# Projections

In [5]:
# load packages
import xarray as xr
import numpy as np
import pandas as pd

# interact with operating system
import os

cwd = os.getcwd()
print("Current working directory:", cwd) 

Current working directory: /home/kim/smb_emulator-main


## Exploring projections of RACMO2.3p2

In [6]:
# define path to data (e.g. stored locally on Roger)
# run `sh downloads.sh` from command line to download data again
full_path = "/home/kim/data/zenodo_7760491_RACMO2-3p2/smb_monthlyS_ANT27_ERA5-3H_RACMO2.3p2_197901_202212.nc"

# open nc file as xarray dataset
ds = xr.open_dataset(full_path)
ds

<xarray.Dataset>
Dimensions:       (time: 528, nblock1: 40, nblock2: 400, bnds: 2, rlat: 240,
                   rlon: 262, height: 1)
Coordinates:
  * height        (height) float64 0.0
    lat           (rlat, rlon) float64 ...
    lon           (rlat, rlon) float64 ...
  * rlat          (rlat) float64 -30.0 -29.75 -29.5 -29.25 ... 29.25 29.5 29.75
  * rlon          (rlon) float64 -32.75 -32.5 -32.25 -32.0 ... 32.0 32.25 32.5
  * time          (time) datetime64[ns] 1979-01-16 ... 2022-12-16
Dimensions without coordinates: nblock1, nblock2, bnds
Data variables:
    assigned      (time) int32 ...
    block1        (nblock1) int32 ...
    block2        (nblock2) int32 ...
    date_bnds     (time, bnds) int32 ...
    dir           (rlat, rlon) float64 ...
    dtg           (time) int32 ...
    hms_bnds      (time, bnds) int32 ...
    rotated_pole  float32 ...
    smb           (time, height, rlat, rlon) float32 ...
    time_bnds     (time, bnds) datetime64[ns] ...
Attributes:
    Conventions:   CF-1.4
    source:        RACMO2
    Domain:        ANT27
    Experiment:    ERA5-3H_RACMO2.3p2
    institution:   Royal Netherlands Meteorological Institute (KNMI)
    CreationDate:  Fri Apr  3 22:43:36 2020
    comment:       asim2cdf: cpar=smb, iwmo=156, ilvt=105, ilev=5, idh=24, ih...
    title:          Surface Mass Balance ice sheet
    NCO:           netCDF Operators version 4.9.7 (Homepage = http://nco.sf.n...
    history:       Fri Mar 10 13:12:49 2023: ncrcat -O /ec/res4/scratch/rumw/...

In [7]:
# inspect the projection of RACMO data
ds.rotated_pole

<xarray.DataArray 'rotated_pole' ()>
[1 values with dtype=float32]
Attributes:
    grid_mapping_name:          rotated_latitude_longitude
    grid_north_pole_latitude:   -180.0
    grid_north_pole_longitude:  -170.0
    proj4_params:               -m 57.295779506 +proj=ob_tran +o_proj=latlon ...
    proj_parameters:            -m 57.295779506 +proj=ob_tran +o_proj=latlon ...
    projection_name:            rotated_latitude_longitude
    long_name:                  projection details
    EPSG_code:

RACMO data is provided on a rotated pole projection. 

Different CORDEX domains have different rotations.

proj4_params :
-m 57.295779506 +proj=ob_tran +o_proj=latlon +o_lat_p=-180.0 +lon_0=10.0

- cartopy.crs.CRS is the class for coordinate reference systems (crs)
  - parameters: proj4_params
    - Defined by defining the new pole
    - '+proj=ob_tran' (general) oblique transformation
    - '+o_proj' projection to be used (order matters)
    - '+o_lat_p' Latitude of the North pole of the unrotated source CRS, expressed in the rotated geographic CRS.
    - '+o_lon_p' Longitude of the North pole of the unrotated source CRS, expressed in the rotated geographic CRS.
    - ['+lon_0' Longitude of projection center.] Default: 0.0 The default convention is to interpret this value as decimal degrees.

[Source of proj details](https://proj.org/en/9.2/operations/projections/ob_tran.html)
[Tool to check projections](https://agrimetsoft.com/Cordex%20Coordinate%20Rotation)
[Notebook by Stef Lhermitte](https://gitlab.tudelft.nl/slhermitte/manuals/-/blob/master/RACMO_reproject.md)
rotated_latitude_longitude

RotPole (13.08; -6.08) (lon, lat)
- The new [rlat, rlon] == [0.0, 0.0] is at [-90.0,	-170.0] which is [90S, 170W]

## Other projections

[WGS 84 / Antarctic Polar Stereographic](https://epsg.io/3031)

    # PROJECTION
    # define projection from rotated pole to APS
    # always x y or lon lat order
    rotatedpole_to_APS71_func = pyproj.Transformer.from_crs(crs_from = pyproj.CRS("-m 57.295779506 +proj=ob_tran +o_proj=latlon +o_lat_p=-180.0 +lon_0=10.0"),  
                                                  crs_to = pyproj.CRS("+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs"), 
                                                  always_xy = True)

        # create meshgrid from rotated_pole grid
    rlon_2d, rlat_2d = np.meshgrid(dataset_smb_xr.rlon.values, dataset_smb_xr.rlat.values)
    # transform points
    x1_2d, x2_2d = rotatedpole_to_APS71_func.transform(rlon_2d, rlat_2d)

In [8]:
lon_df = xr.DataArray.to_dataframe(ds.coords["lon"], name = "lon")
lon_df[lon_df["lat"] < -89.]

lat         lon         lon
rlat  rlon                                    
-0.75 -0.50 -89.098620 -136.311775 -136.311775
      -0.25 -89.209433 -151.566078 -151.566078
       0.00 -89.250000 -170.000000 -170.000000
       0.25 -89.209433  171.566078  171.566078
       0.50 -89.098620  156.311775  156.311775
-0.50 -0.75 -89.098620 -113.691498 -113.691498
      -0.50 -89.292897 -125.001091 -125.001091
      -0.25 -89.440984 -143.435594 -143.435594
       0.00 -89.500000 -170.000000 -170.000000
       0.25 -89.440984  163.435594  163.435594
       0.50 -89.292897  145.001091  145.001091
       0.75 -89.098620  133.691498  133.691498
-0.25 -0.75 -89.209433  -98.435558  -98.435558
      -0.50 -89.440984 -106.565497 -106.565497
      -0.25 -89.646447 -125.000273 -125.000273
       0.00 -89.750000 -170.000000 -170.000000
       0.25 -89.646447  145.000273  145.000273
       0.50 -89.440984  126.565497  126.565497
       0.75 -89.209433  118.435558  118.435558
 0.00 -0.75 -89.250000  -80.000016  -80.000016
      -0.50 -89.500000  -80.000024  -80.000024
      -0.25 -89.750000  -80.000047  -80.000047
       0.00 -90.000000 -170.000000 -170.000000
       0.25 -89.750000  100.000047  100.000047
       0.50 -89.500000  100.000024  100.000024
       0.75 -89.250000  100.000016  100.000016
 0.25 -0.75 -89.209433  -61.564470  -61.564470
      -0.50 -89.440984  -53.434541  -53.434541
      -0.25 -89.646447  -34.999774  -34.999774
       0.00 -89.750000    9.999965    9.999965
       0.25 -89.646447   54.999774   54.999774
       0.50 -89.440984   73.434541   73.434541
       0.75 -89.209433   81.564470   81.564470
 0.50 -0.75 -89.098620  -46.308524  -46.308524
      -0.50 -89.292898  -34.998933  -34.998933
      -0.25 -89.440984  -16.564425  -16.564425
       0.00 -89.500000   10.000000   10.000000
       0.25 -89.440984   36.564425   36.564425
       0.50 -89.292898   54.998933   54.998933
       0.75 -89.098620   66.308524   66.308524
 0.75 -0.50 -89.098620  -23.688240  -23.688240
      -0.25 -89.209433   -8.433931   -8.433931
       0.00 -89.250000   10.000000   10.000000
       0.25 -89.209433   28.433931   28.433931
       0.50 -89.098620   43.688240   43.688240
 1.00  0.00 -89.000000   10.000000   10.000000

# Check McMurdo

GEO URI  
latitude: -77.8  
longitude: 166.6

In [10]:
lon_df = xr.DataArray.to_dataframe(ds.coords["lon"], name = "lon")
# remove duplicate column
lon_df = lon_df.iloc[:, :-1]
lon_df[(lon_df["lat"] < -76.0) & (lon_df["lat"] > -79.0) & (lon_df["lon"] < 168.0) & (lon_df["lon"] > 165.0)]
#lon_df[(lon_df["lat"] < -76.0) & (lon_df["lat"] > -79.0)]
# lon_df[(lon_df["lon"] < 168.0) & (lon_df["lon"] > 165.0)]

lat         lon
rlat   rlon                       
-12.75 5.25 -76.227989  167.982627
       5.50 -76.132368  167.043588
       5.75 -76.033000  166.117793
-12.50 5.25 -76.458450  167.572242
       5.50 -76.361149  166.619614
       5.75 -76.260061  165.680878
-12.25 5.25 -76.688219  167.147938
       5.50 -76.589188  166.181477
       5.75 -76.486327  165.229593
-12.00 5.00 -77.014115  167.704628
       5.25 -76.917260  166.709027
       5.50 -76.816445  165.728490
-11.75 5.00 -77.244183  167.265468
       5.25 -77.145533  166.254778
       5.50 -77.042879  165.259928
-11.50 4.75 -77.569852  167.852922
       5.00 -77.473499  166.810474
       5.25 -77.372998  165.784414
-11.25 4.75 -77.800239  167.397795
       5.00 -77.702023  166.338817
       5.25 -77.599609  165.297111
-11.00 4.75 -78.029850  166.925447
       5.00 -77.929708  165.849612
-10.75 4.50 -78.356365  167.546847
       4.75 -78.258641  166.434932
       5.00 -78.156507  165.341918
-10.50 4.50 -78.586293  167.055728
       4.75 -78.486562  165.925237
-10.25 4.25 -78.912537  167.714939
       4.50 -78.815371  166.544726
       4.75 -78.713562  165.395280